# Opening and Exploring FITS files

## FITS files are the standard astronomical image format
At some point, probably fairly quickly in your astronomy career, you will encounter FITS files. 99 times out of 100, they will be astronomical images. That last 1% is reserved for those special occasions when a FITS file is actually a table. This case will be dealt with in another example.

For now, lets look at what this example will show you. It will:

* Show you how to load in a FITS file and see what it contains
* Go through FITS headers, keywords, and what they mean
* Have a look at FITS data, and briefly show what you can do with it (data manipulation and presentation will be covered in other examples)

So, lets get started. 

## Introducing AstroPy - A Python Astronomers Best Friend! (And Worst Nightmare!)
It should come as a surprise to noone that, as with anything that Python can be used for, astronomy has a thriving Python community, and one of the largest and most well used modules is AstroPy. AstroPy is extremely versetile, packed full of useful tools, many of which will be covered in other examples and tutorials. As such, the AstroPy [documentation](https://docs.astropy.org/en/stable/index.html) is extensive and very, very through. This is both a blessing and a curse. All the information you could ever want is in there... somewhere. Knowing where is an art in and of itself. You will get there, dont worry! 

This tutorial is concerned with reading in and writing out FITS files. This is handled by the ``astropy.io.fits`` module. And you can go to the AstroPy documentation and find the section relating to [just FITS file handling](https://docs.astropy.org/en/stable/io/fits/index.html) and look up everything you need there. But there is a **LOT** of information to take in. So this example goes over just a fraction of it. Just the basics of taking a first look at a FITS file.

## Importing AstroPy's input/output FITS module and opening a FITS file
This example comes, rather handily, with its own little FITS image file. AstroPy has some example files too, but having one on disc is rather nice. It's a very simple file, designed to mimic a *Herschel* SPIRE photometry image, probably in the 350 micon band. Dont worry about this too much. It could be literally any image. 

The first thing we need to do is import the right module.

As a quick note, this example will be using Python 3.8.3, and Anaconda 2020.07, which comes with AstroPy 4.0.1. BUT, this code should work with Python 2.7 and Astropy 2 or better. Really, you sould try to switch to Python 3, but a lot of astronomy is stuck in the mud, and still uses Python 2.7. It's not a big deal, though there are a few key differences. If you ask for help online, most people will assume you are using Python 3.

In [4]:
from astropy.io import fits # We are just importing the fits module from astropy

In [6]:
# Define path to the example fits file
path_to_file = './Data/ExampleFITS.fits'

# Read in the fits file with fits.open
# HDU is the term used for a Header Data Unit - a "thing" that contains both "header" information and "data" information
# FITS files can contain multiple HDUs, composed into a List.
HDUList = fits.open(path_to_file)

# Lets check out what the HDUList contains!
HDUList.info()

Filename: ./Data/ExampleFITS.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (500, 500)   float64   


The ``.info`` method prints out all the information from the HDU List. In general, each HDU contains some information regarding the file. These may be images or image cubes (or even hypercubes with more than 3 dimensions!) with associated headers which tell you what each image contains, or they may be headers without images which contain information regarding the file itself, such as when it was created, or what telescope was used. Sometimes, an HDU might be something like a table. Dont worry if some of this is confusing, it will be explained in a moment.

For now, it is just important to note that when opened this way, HDU Lists can contain more than one HDU, and some of those might be image/header pairs. These pairs may each tell a different part of the story. The first pair is usually the *data* pair. This is the actual observation. The second pair may be a corresponding map of the noise, or some other secondary information. 

It is also worth noting that sometimes, while the first image/header pairing in an HDU List SHOULD be the *data*, the PrimaryHDU is not always an image/header pair. Instead, it is often just a header, giving overall information about the observing run. So just be careful. If you use the ``.info`` method and ``PRIMARY`` does not have any Dimensions listed, then there is no image data there.

Lets go back to our example FITS file. In our case, there is only one HDU, and it appears to contain an image (as shown by the Dimensions having a physical size) and a header (shown by the number of Cards). So lets select this HDU and see what it contains.

In [7]:
# Save the first (and in this case, only) element of HDUList to a new variable
HDU = HDUList[0]

# Now print out the header
HDU.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  500                                                  
NAXIS2  =                  500                                                  
BUNIT   = 'MJy/sr  '           / Unit of the data                               
CRPIX1  =                249.5 / [] WCS: Reference pixel position axis 1        
CRPIX2  =                249.5 / [] WCS: Reference pixel position axis 2        
CRVAL1  =                  0.0 / [] WCS: First coordinate of reference pixel    
CRVAL2  =                  0.0 / [] WCS: Second coordinate of referene pixel    
CDELT1  = -0.00027777777777777 / [] WCS: Pixel scale axis 1, unit=Angle         
CDELT2  = 0.000277777777777777 / [] WCS: Pixel scale axis 2, unit=Angle         
CTYPE1  = 'RA---TAN'        